In [92]:
import pandas as pd
import numpy as np
import operator
from scipy.spatial.distance import euclidean

In [93]:
def remove_small_probabilities(df, threashold = 0.00055):
    return df[df['probability']>threashold]

def assign_square(df, square_size):
    df['square_lat'] = (df['lat']/square_size).astype(int)*square_size
    df['square_lng'] = (df['lng']/square_size).astype(int)*square_size
    return df

def calc_square_representative(df):
    representative = {}
    grouped = df.groupby(['square_lat', 'square_lng'])
    for g in grouped.groups: representative[g]=np.mean(grouped.get_group(g)['probability'])
    return representative

In [94]:
spreading_df = pd.read_csv('data/spreading/spreading.csv')
start = spreading_df.loc[0]
spreading_df.drop(0, axis=0, inplace=True)
spreading_df = remove_small_probabilities(spreading_df)
#normalize lat and lng
lat_dom = min(spreading_df['lat'])
lng_dom = min(spreading_df['lng'])
spreading_df['lat'] = spreading_df['lat']-min(spreading_df['lat'])
spreading_df['lng'] = spreading_df['lng']-min(spreading_df['lng'])
#maping years from 10 to 5
spreading_df = spreading_df[spreading_df['year']<10]
spreading_df['year'] = ((spreading_df['year']+2)/2).astype(int)

In [95]:
#calculate for each data point its square, and probability of its square 
SQUARE_SIZE= 5
spreading_df = assign_square(spreading_df, SQUARE_SIZE)
square_prob= calc_square_representative(spreading_df)
def map_prob_sqr(sqr_lat, sqr_lng):
    return square_prob[(sqr_lat, sqr_lng)]
spreading_df['square_prob'] = np.vectorize(map_prob_sqr)(spreading_df['square_lat'], spreading_df['square_lng'])

In [96]:
def get_closest_start(point, level_points):
    min_dist = 100000
    for node in level_points:
        if euclidean(np.array([node[0], node[1]]), np.array(point)) <min_dist:
            min_node = node
            min_dist = euclidean(np.array([node[0], node[1]]), np.array(point))
    return min_node

In [97]:
CHILD_THRESHOLD = 0.001
grouped_years = spreading_df.groupby(['year', 'square_lat', 'square_lng'])
tree = {(start['lat'], start['lng'],0): []}
levels =[[]]*6
levels[0] = [(start['lat'], start['lng'], 0)]
spreading_df = pd.DataFrame(grouped_years.sum())
for g in grouped_years.groups:
    if (g[1], g[2]) in tree.keys(): continue
    if grouped_years.get_group(g)['probability'].any() >=CHILD_THRESHOLD: 
        g = list(g)
        g[1]+=lat_dom+5
        g[2]+=lng_dom+5
        parent = get_closest_start((g[1],g[2]),levels[g[0]-1])
        levels[g[0]].append((g[1], g[2], g[0]))
        tree[parent].append((g[1],g[2],g[0]))
        tree[(g[1],g[2], g[0])] = []   

In [98]:
nodes_df = pd.DataFrame({'node_lng': [i[1] for i in tree.keys()], 'node_lat': [i[0] for i in tree.keys()], 'depth': [i[2] for i in tree.keys()]})
nodes_df.to_csv('data/spreading/spreading_tree_nodes.csv')
edges_df=pd.DataFrame(columns = ['parent_lat', 'parent_lng', 'child_lat', 'child_lng', 'depth'])
for p in tree.keys():
    edges_df = edges_df.append(pd.DataFrame({'parent_lat': p[0], 'parent_lng':p[1], 'depth': [i[2] for i in tree[p]],\
                                    'child_lat': [i[0] for i in tree[p]], 'child_lng': [i[1] for i in tree[p]]}))
edges_df.to_csv('data/spreading/spreading_tree_edges.csv')

In [99]:
edges_df

,child_lat,child_lng,depth,parent_lat,parent_lng
0,-37,24,1,-11.0,43.0
1,-37,34,1,-11.0,43.0
2,-32,29,1,-11.0,43.0
3,-32,34,1,-11.0,43.0
4,-32,39,1,-11.0,43.0
5,-27,34,1,-11.0,43.0
6,-27,39,1,-11.0,43.0
7,-27,44,1,-11.0,43.0
8,-22,34,1,-11.0,43.0
9,-22,39,1,-11.0,43.0


In [100]:
nodes_df

,depth,node_lat,node_lng
0,0,-11.0,43.0
1,1,-37.0,24.0
2,1,-37.0,34.0
3,1,-32.0,29.0
4,1,-32.0,34.0
5,1,-32.0,39.0
6,1,-27.0,34.0
7,1,-27.0,39.0
8,1,-27.0,44.0
9,1,-22.0,34.0
